# Demo of SED and photometry calculations with DSPS
author: Luca Tortorelli, Andrew Hearin

last run successfully: Nov 7, 2022

This notebook demonstrates some basic usage of the DSPS library. In particular, for a galaxy with some arbitrary star formation history, we'll see how to calculate its restframe SED, and its absolute and apparent magnitude in some band.

Andrew:
Warning: At present, DSPS is mostly just a collection of JAX kernels and does not have much in the way of a user-interface or documentation. The API shown here is very likely to evolve in the coming year. I expect that a stable interface with end-user docs will not arrive until spring 2023 at the earliest.

SPS calculations are based on a set of template SEDs of simple stellar populations (SSPs). Supplying such templates is outside the planned scope of the DSPS package, and so they will need to be retrieved from some other library. For example, the FSPS library supplies such templates in a convenient form.

In [ ]:
import numpy as np
import h5py
import os
from rail.core.stage import RailStage
from src.rail.creation.engines.dsps_sed_modeler import DSPSSingleSedModeler, DSPSPopulationSedModeler
from src.rail.creation.engines.dsps_photometry_creator import DSPSPhotometryCreator
from rail.core.utils import RAILDIR
import ceci

In [ ]:
DS = RailStage.data_store
DS.__class__.allow_overwrite = True

The SSP library that is required to generate an SED model is stored in /src/rail/examples_data/creation_data/data/dsps_default_data. Dummy input galaxy population properties to be used by DSPS to create galaxy SEDs can be generated via this notebook. The required data are:
- the fluxes of the single stellar populations (SSP)
- the corresponding SSP wavelengths
- the age grid of the SSP in units of log10(Age[Gyr])
- the metallicity grid of the SSP in units of log10(Z*/Z*_solar)
- the galaxy star-formation history in units of log10(M*/M*_solar) per time step in units of Gyr
- the cosmic time age grid up to the Universe age in Gyr
- galaxy age at the time of observation in Gyr
- galaxy metallicity at the time of observation in log10(Z*/Z*_solar)
- log normal scatter of the galaxy metallicity at the time of observation

To better clarify the meaning of the parameters above, we provide an example of how to generate a dummy hdf5 input galaxy properties table.

In [ ]:
default_files_folder = os.path.join(RAILDIR, 'rail', 'examples_data', 'creation_data', 'data', 'dsps_default_data')

In [ ]:
n_galaxies = 10

redshift = np.arange(0.1, 1.1, 0.1)

gal_t_table = np.linspace(0.05, 13.8, 100) # age of the universe in Gyr
gal_sfr_table = np.random.uniform(0, 10, gal_t_table.size) # SFR in Msun/yr

gal_lgmet = -2.0 # log10(Z)
gal_lgmet_scatter = 0.2 # lognormal scatter in the metallicity distribution function

cosmic_time_grid = np.full((n_galaxies, len(gal_t_table)), gal_t_table)
star_formation_history = np.full((n_galaxies, len(gal_sfr_table)), gal_sfr_table)
stellar_metallicity = np.full(n_galaxies, gal_lgmet)
stellar_metallicity_scatter = np.full(n_galaxies, gal_lgmet_scatter)

with h5py.File(os.path.join(default_files_folder, 'input_galaxy_properties_dsps.h5'), 'w') as h5table:
    h5table.create_dataset(name='redshift', data=redshift)
    h5table.create_dataset(name='cosmic_time_grid', data=cosmic_time_grid)
    h5table.create_dataset(name='star_formation_history', data=star_formation_history)
    h5table.create_dataset(name='stellar_metallicity', data=stellar_metallicity)
    h5table.create_dataset(name='stellar_metallicity_scatter', data=stellar_metallicity_scatter)

### Class initialization for the creation of single SED models with DSPS

The parallelization in this class is done using MPI

In [ ]:
single_sed_model = DSPSSingleSedModeler.make_stage(name='DSPS_single_SED_model',
                                                   ssp_templates_file=os.path.join(RAILDIR,
                                                                                   'rail/examples_data/creation_data/data/dsps_default_data/ssp_data_fsps_v3.2_lgmet_age.h5'),
                                                   redshift_key='redshift',
                                                   cosmic_time_grid_key='cosmic_time_grid',
                                                   star_formation_history_key='star_formation_history',
                                                   stellar_metallicity_key='stellar_metallicity',
                                                   stellar_metallicity_scatter_key='stellar_metallicity_scatter',
                                                   restframe_sed_key='restframe_seds')

Passing input galaxy properties

In [ ]:
trainFile = os.path.join(default_files_folder, 'input_galaxy_properties_dsps.h5')
h5table = h5py.File(trainFile, 'r')
single_sed_model.add_data('input', h5table)

Run() and fit_model() to generate the actual rest-frame SED in units of L_solar/Hz. Since the output is a Hdf5Handle, the rest-frame SED is stored into a .hdf5 file

In [ ]:
single_sed_model.fit_model()
h5table.close()

In [ ]:
rest_frame_sed_models = single_sed_model.get_data('model')
print(rest_frame_sed_models)

### Class initialization for the creation of single SED models with DSPS using Ceci

In [ ]:
DSPS_single_SED_model = DSPSSingleSedModeler.make_stage(name='DSPS_single_SED_model',
                                                        ssp_templates_file=os.path.join(RAILDIR, 'rail/examples_data/creation_data/data/dsps_default_data/ssp_data_fsps_v3.2_lgmet_age.h5'),
                                                        redshift_key='redshift',
                                                        cosmic_time_grid_key='cosmic_time_grid',
                                                        star_formation_history_key='star_formation_history',
                                                        stellar_metallicity_key='stellar_metallicity',
                                                        stellar_metallicity_scatter_key='stellar_metallicity_scatter',
                                                        restframe_sed_key='restframe_seds')

In [ ]:
pipe = ceci.Pipeline.interactive()
stages = [DSPS_single_SED_model]
for stage in stages:
    pipe.add_stage(stage)

In [ ]:
trainFile = os.path.join(default_files_folder, 'input_galaxy_properties_dsps.h5')
pipe.initialize(dict(input=trainFile), dict(output_dir='./temp_output_dspsv3.0.0', log_dir='./logs_dspsv3.0.0',
                                            resume=False, nprocess=2), None)

In [ ]:
pipe.save('./temp_output_dspsv3.0.0/pipe_saved.yml')
pr = ceci.Pipeline.read('./temp_output_dspsv3.0.0/pipe_saved.yml')
pr.run()

### Class initialization for the creation of a population of SEDs with DSPS

In case we would like to generate a population of model rest-frame SEDs, we need to invoke the DSPSPopulationSedModeler class. In this case, the parallelization is managed by JAX.

In [ ]:
DSPS_population_SED_model = DSPSPopulationSedModeler.make_stage(name='DSPS_population_SED_model',
                                                                ssp_templates_file=os.path.join(RAILDIR, 'rail/examples_data/creation_data/data/dsps_default_data/ssp_data_fsps_v3.2_lgmet_age.h5'),
                                                                redshift_key='redshift',
                                                                cosmic_time_grid_key='cosmic_time_grid',
                                                                star_formation_history_key='star_formation_history',
                                                                stellar_metallicity_key='stellar_metallicity',
                                                                stellar_metallicity_scatter_key='stellar_metallicity_scatter',
                                                                restframe_sed_key='restframe_seds')

In [ ]:
trainFile = trainFile = os.path.join(default_files_folder, 'input_galaxy_properties_dsps.h5')
h5table = h5py.File(trainFile, 'r')
DSPS_population_SED_model.add_data('input', h5table)

In [ ]:
DSPS_population_SED_model.fit_model()
h5table.close()

In [ ]:
rest_frame_sed_models = DSPS_population_SED_model.get_data('model')
print(rest_frame_sed_models)

### Class initialization for the creation of a population of SEDs with DSPS using ceci


In [ ]:
DSPS_population_SED_model = DSPSPopulationSedModeler.make_stage(name='DSPS_population_SED_model',
                                                                ssp_templates_file=os.path.join(RAILDIR, 'rail/examples_data/creation_data/data/dsps_default_data/ssp_data_fsps_v3.2_lgmet_age.h5'),
                                                                redshift_key='redshift',
                                                                cosmic_time_grid_key='cosmic_time_grid',
                                                                star_formation_history_key='star_formation_history',
                                                                stellar_metallicity_key='stellar_metallicity',
                                                                stellar_metallicity_scatter_key='stellar_metallicity_scatter',
                                                                restframe_sed_key='restframe_seds')

In [ ]:
pipe = ceci.Pipeline.interactive()
stages = [DSPS_population_SED_model]
for stage in stages:
    pipe.add_stage(stage)

In [ ]:
pipe.initialize(dict(input=trainFile), dict(output_dir='./temp_output_dspsv3.0.0', log_dir='./logs_dspsv3.0.0',
                                            resume=False, nprocess=2), None)

In [ ]:
pipe.save('./temp_output_dspsv3.0.0/pipe_saved.yml')
pr = ceci.Pipeline.read('./temp_output_dspsv3.0.0/pipe_saved.yml')
pr.run()

### Observed photometry generation with DSPS

Generating the observed photometry with DSPS is simple and requires only few input from the user. The required data are:
- filter wavebands
- instrument name
- rest-frame sed models
- galaxy redshifts
- wCDM cosmological parameters

In [ ]:
DSPS_photometry_creator = DSPSPhotometryCreator.make_stage(name='DSPS_photometry_creator',
                                                           redshift_key='redshift',
                                                           restframe_sed_key='restframe_seds',
                                                           absolute_mags_key='rest_frame_absolute_mags',
                                                           apparent_mags_key='apparent_mags',
                                                           filter_folder=os.path.join(RAILDIR, 'rail/examples_data/creation_data/data/dsps_default_data/filters'),
                                                           instrument_name='lsst',
                                                           wavebands='u,g,r,i,z',
                                                           ssp_templates_file=os.path.join(RAILDIR, 'rail/examples_data/creation_data/data/dsps_default_data/ssp_data_fsps_v3.2_lgmet_age.h5'))

In [ ]:
trainFile = os.path.join(default_files_folder, 'model_DSPS_single_SED_model.hdf5')
h5table = h5py.File(trainFile, 'r')
DSPS_photometry_creator.add_data('model', h5table)

The sample() method computes the absolute magnitudes and the observed magnitudes in the AB system for all the provided filters and for all the galaxies and store the results in a hdf5 table

In [ ]:
output_mags = DSPS_photometry_creator.sample()
h5table.close()

In [ ]:
print(output_mags.data)

### Observed photometry generation with DSPS using ceci

In [ ]:
DSPS_photometry_creator = DSPSPhotometryCreator.make_stage(name='DSPS_photometry_creator',
                                                           redshift_key='redshifts',
                                                           restframe_sed_key='restframe_seds',
                                                           absolute_mags_key='rest_frame_absolute_mags',
                                                           apparent_mags_key='apparent_mags',
                                                           filter_folder=os.path.join(RAILDIR, 'rail/examples_data/creation_data/data/dsps_default_data/filters'),
                                                           instrument_name='lsst',
                                                           wavebands='u,g,r,i,z',
                                                           ssp_templates_file=os.path.join(RAILDIR, 'rail/examples_data/creation_data/data/dsps_default_data/ssp_data_fsps_v3.2_lgmet_age.h5'))

In [ ]:
pipe = ceci.Pipeline.interactive()
stages = [DSPS_photometry_creator]
for stage in stages:
    pipe.add_stage(stage)

In [ ]:
trainFile = os.path.join(default_files_folder, 'model_DSPS_single_SED_model.hdf5')
pipe.initialize(dict(model=trainFile), dict(output_dir='./temp_output_dspsv3.0.0', log_dir='./logs_dspsv3.0.0',
                                            resume=False, nprocess=2), None)

In [ ]:
pipe.save('./temp_output_dspsv3.0.0/pipe_saved.yml')
pr = ceci.Pipeline.read('./temp_output_dspsv3.0.0/pipe_saved.yml')
pr.run()